In [96]:
from fast_to_SQL.fast_to_SQL import to_sql_fast
import pandas as pd
import numpy as np
from urllib.parse import quote_plus
import sqlalchemy

params = quote_plus(
    """DRIVER={ODBC Driver 13 for SQL Server};
    SERVER=server-dw01.database.windows.net;
    DATABASE=DataWarehouseStaging;
    UID=dw_userlogin;
    PWD=kljDueK!7"""
)

ENGINE = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [98]:
TO_df = pd.read_sql('''SELECT [com_id_h]
    ,[Opportunity_ID]
    ,[First_of_Decision_Month]
	,[-12]
	,[-11]
	,[-10]
	,[-9]
	,[-8]
	,[-7]
	,[-6]
	,[-5]
	,[-4]
	,[-3]
	,[-2]
	,[-1]

FROM [pam].[TO_dataset_relative]

''', ENGINE)

In [99]:
trans_df = pd.read_sql('''SELECT [com_id_h]
    ,[Opportunity_ID]
    ,[First_of_Decision_Month]
	,[-12]
	,[-11]
	,[-10]
	,[-9]
	,[-8]
	,[-7]
	,[-6]
	,[-5]
	,[-4]
	,[-3]
	,[-2]
	,[-1]

FROM [pam].[trans_dataset_relative]

''', ENGINE)

In [100]:
feature_df = pd.read_sql('''SELECT * FROM pam.dataset_features_dedupe''', ENGINE)

In [101]:
def rename_df_cols(df, prefix):
    col_list = list(df.columns)
    col_list = [i for i in col_list if i not in ['com_id_h', 'First_of_Decision_Month', 'Opportunity_ID']]
    new_col_list = [prefix + i.replace("-", "") for i in col_list]
    col_rename_dict = dict(zip(col_list, new_col_list))
    return df.rename(columns = col_rename_dict)

In [102]:
TO_df_renamed = rename_df_cols(TO_df, 'turnover_month_')

In [103]:
trans_df_renamed = rename_df_cols(trans_df, 'trans_num_month_')

In [104]:
vols_df = TO_df_renamed.merge(trans_df_renamed, how = 'inner', on = ['com_id_h', 'First_of_Decision_Month', 'Opportunity_ID'])

In [105]:
df = vols_df.merge(feature_df, how = 'left', left_on = ['com_id_h', 'First_of_Decision_Month'], right_on = ['com_id_h', 'file_date'])

In [106]:
recent_df = pd.read_sql('''SELECT *

FROM pam.dataset_features_dedupe AS d

WHERE file_date = (
		SELECT MAX(file_date)
		
		FROM pam.dataset_features_dedupe AS exc
		
		WHERE exc.com_id_h = d.com_id_h
		)

''', ENGINE)

In [107]:
calc_df = df.merge(recent_df, how = 'left', on = 'com_id_h')

In [108]:
calc_df['month_diff'] = ((calc_df['file_date_y'] - calc_df['First_of_Decision_Month'])/np.timedelta64(1, 'M')).astype(int)

In [109]:
calc_df['calc_tenure_in_months'] = calc_df['tenure_in_months_y'] - calc_df['month_diff']

In [110]:
calc_df['tenure_in_months'] = calc_df['tenure_in_months_x'].combine_first(calc_df['calc_tenure_in_months'])

In [111]:
calc_df.head()

,com_id_h,Opportunity_ID,First_of_Decision_Month,turnover_month_12,turnover_month_11,turnover_month_10,turnover_month_9,turnover_month_8,turnover_month_7,turnover_month_6,...,trans_num_month_1,file_date_x,MCC_DESC_x,tenure_in_months_x,file_date_y,MCC_DESC_y,tenure_in_months_y,month_diff,calc_tenure_in_months,tenure_in_months
0,000A58863896B7F82B41C2AE73C152D40F27A9D1000000...,006w000000p3sZvAAI,2017-03-01,NaN,5644.000,4410.00,4725.00,5675.0,7586.00,4945.000,...,NaN,NaT,NaN,NaN,2019-11-01,DRINKING PLACES (ALCOHOLIC),59.0,32,27.0,27.0
1,00174CEAF374646AEFC22F47F096A4265B313564000000...,0062X00000wd0b7QAA,2018-12-01,17348.93,32110.210,45740.00,26550.00,39880.0,48185.00,45719.000,...,58.0,2018-12-01,MEDICAL SERVICES MISC,171.0,2019-11-01,MEDICAL SERVICES MISC,182.0,11,171.0,171.0
2,0025505E3307957A9646C27BBBF7FB827EA22A4D000000...,0062X00000upQYdQAM,2018-04-01,19120.00,25541.000,27353.25,24406.35,29072.1,24235.30,23380.745,...,132.0,NaT,NaN,NaN,2019-11-01,HOTELS MOTELS & RESORTS-LODGIN,66.0,19,47.0,47.0
3,0042E63ECC3854929AC9D3A357703253ADE83198000000...,0062X00000x96aqQAA,2019-01-01,16606.67,17492.460,20162.44,15979.98,14616.5,21994.11,17860.850,...,19.0,2019-01-01,FREIGHT LOCAL DELIVERY REMOVAL,81.0,2019-11-01,FREIGHT LOCAL DELIVERY REMOVAL,91.0,9,82.0,81.0
4,0045F0C43EE1D16503D9DE215E9EDB3884FB92E1000000...,0062X00000vYL3aQAG,2018-07-01,2901.00,4168.325,3222.15,2466.00,2727.0,875.00,3443.000,...,18.0,2018-07-01,MENS & WOMENS CLOTHING STORES,55.0,2019-11-01,MENS & WOMENS CLOTHING STORES,71.0,16,55.0,55.0


In [112]:
calc_df['MCC_DESC'] = calc_df['MCC_DESC_x'].combine_first(calc_df['MCC_DESC_y'])

In [113]:
calc_df.columns

Index(['com_id_h', 'Opportunity_ID', 'First_of_Decision_Month',
       'turnover_month_12', 'turnover_month_11', 'turnover_month_10',
       'turnover_month_9', 'turnover_month_8', 'turnover_month_7',
       'turnover_month_6', 'turnover_month_5', 'turnover_month_4',
       'turnover_month_3', 'turnover_month_2', 'turnover_month_1',
       'trans_num_month_12', 'trans_num_month_11', 'trans_num_month_10',
       'trans_num_month_9', 'trans_num_month_8', 'trans_num_month_7',
       'trans_num_month_6', 'trans_num_month_5', 'trans_num_month_4',
       'trans_num_month_3', 'trans_num_month_2', 'trans_num_month_1',
       'file_date_x', 'MCC_DESC_x', 'tenure_in_months_x', 'file_date_y',
       'MCC_DESC_y', 'tenure_in_months_y', 'month_diff',
       'calc_tenure_in_months', 'tenure_in_months', 'MCC_DESC'],
      dtype='object')

In [114]:
final_df = calc_df[['com_id_h', 'Opportunity_ID', 'First_of_Decision_Month', 'tenure_in_months', 'MCC_DESC', 'turnover_month_12',
       'turnover_month_11', 'turnover_month_10', 'turnover_month_9',
       'turnover_month_8', 'turnover_month_7', 'turnover_month_6',
       'turnover_month_5', 'turnover_month_4', 'turnover_month_3',
       'turnover_month_2', 'turnover_month_1', 'trans_num_month_12',
       'trans_num_month_11', 'trans_num_month_10', 'trans_num_month_9',
       'trans_num_month_8', 'trans_num_month_7', 'trans_num_month_6',
       'trans_num_month_5', 'trans_num_month_4', 'trans_num_month_3',
       'trans_num_month_2', 'trans_num_month_1']]

In [115]:
to_sql_fast(final_df, name = 'pam.dataset_PAMready', engine = ENGINE, if_exists = 'replace')